In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.reset_default_graph()

In [0]:
char_arr = [c for c in 'SEPhelo']
num_dic = {n:i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

input_batch = []
target_batch =[]

In [0]:
char_arr

['S', 'E', 'P', 'h', 'e', 'l', 'o']

In [0]:
num_dic

{'E': 1, 'P': 2, 'S': 0, 'e': 4, 'h': 3, 'l': 5, 'o': 6}

In [0]:
input1 = [num_dic[n] for n in 'hell']
target1 = [num_dic[n] for n in 'ello'] # 트레이닝 시
target2 = [num_dic[n] for n in 'PPPP'] # 테스트 시
#input_batch.append(np.eye(dic_len)[input1])
#target_batch.append(np.eye(dic_len)[target1])

In [0]:
target1

[4, 5, 5, 6]

In [0]:
input_batch = np.eye(dic_len)[input1]
target_batch = np.eye(dic_len)[target1]

In [0]:
target_batch

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [0]:
learning_rate = 0.01
n_hidden = 12
#total_epoch = 100
n_step = 4 #최대 4개의 input만 받는
# 입력과 출력의 형태가 one-hot 인코딩으로 같으므로 크기도 같다.
n_class = dic_len
n_input = dic_len

In [0]:
input = tf.placeholder(tf.float32, [1,n_step, n_input], name='input')
target = tf.placeholder(tf.float32, [1, n_step], name='target')

In [0]:
# RNN 셀을 구성한다.
with tf.variable_scope('rnn_cell'):
    rnn_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    #rnn_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)

    outputs, enc_states = tf.nn.dynamic_rnn(rnn_cell, input,
                                            dtype=tf.float32)

In [0]:
model = tf.layers.dense(outputs, n_class, activation=None)
model = tf.reshape(model, [n_step,n_class])
#prediction = tf.argmax(model, 0)
#m = tf.identity(model, name='m')
#p = tf.identity(prediction, name='p')

cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=model, labels=target1))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
model.shape

TensorShape([Dimension(1), Dimension(4), Dimension(4)])

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch = input_batch.reshape([1,n_step, n_input])
target1 = np.reshape(target1, [1,n_step])

for i in range(10000):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={input: input_batch,
                                  target: target1})
    if i%100 == 0:
      print(i, 'cost', '{:.6f}'.format(loss))

In [0]:
prediction = tf.argmax(model, 1)
target2 = np.reshape(target2, [1,n_step])
result = sess.run([prediction],
                       feed_dict={input: input_batch, target: target2})
decoded = [char_arr[i] for i in result[0]]

print(decoded)

['e', 'l', 'l', 'o']
